In [77]:
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q
!pip install torchsummaryX==1.3.0 -q
!pip install scikit-learn matplotlib
!apt-get update && apt-get install -y unzip


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 218.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 158.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 304.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 242.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease                 
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease               
Hit:4 http://se

In [75]:
import torch
import torch.nn as nn

from torchsummaryX import summary
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import gc
import os
import math
import yaml
import random
import json
import sklearn

import glob
import zipfile
import numpy as np
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")


In [ ]:
!unzip '/workspace/concatenated_residue_lig_strings'

In [23]:
 json_file = '/workspace/concatenated_residue_lig_strings.json'

In [ ]:
with open("/workspace/concatenated_residue_strings.json", "r") as file:
    residuedata = json.load(file)

In [68]:
class LigandFeatExtractor(nn.Module):
    def __init__(self):
        super(LigandFeatExtractor, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(1, 5), stride=(1, 1), padding=(0, 0))
        self.conv2 = torch.nn.Conv2d(64, 128, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        self.conv3 = torch.nn.Conv2d(128, 256, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        self.pool = torch.nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1))
        self.flatten = torch.nn.Flatten()
        
        self.fc1 = torch.nn.Linear(101 * 59, 512)
        self.fc2 = torch.nn.Linear(512, 256)
        self.output = torch.nn.Linear(256, 128)  # num_classes for classification
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.output(x)  # Output layer (classification)
        return x


In [37]:
with open("binding_site_coords_as_lists.json", "r") as file:
    data = json.load(file)

In [61]:
numResElems = 101
maxNumResidues = 59

# Assuming each residue has a feature vector with 'num_features' elements
num_features = len(residues[0])  # Get the number of features from the first residue

proteins = []
for prot in data.values():
    residues = [resArr for resArr in prot.values()]
    
    # Pad with a list of zero features for each missing residue
    padding = [0] * num_features  # Zero-padded residue with the same number of features
    while len(residues) < maxNumResidues:
        residues.append(padding)  # Append padding to match maxNumResidues

    proteins.append(residues)


In [70]:
tensorData = torch.Tensor(proteins)

model = LigandFeatExtractor()

from sklearn.model_selection import train_test_split
trainData, testData = 

In [81]:
import torch.optim as optim

# Loss function to maximize variance of the output features
def variance_loss(features):

    feature_variance = torch.var(features, dim=0).mean()  # Mean variance across all features
    return -feature_variance  # Negative variance to maximize variance during training

def train(model, data_loader, epochs=10, lr=0.001, device='cpu'):
    model.to(device)  # Move model to the device (GPU or CPU)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for data in data_loader:
            # Assuming 'data' is a batch of shape (batch_size, 1, 59, 101)
            data = data[0].to(device)  # Move data to the correct device
            optimizer.zero_grad()
            
            # Forward pass
            features = model(data)
            
            # Compute variance loss
            loss = variance_loss(features)
            
            # Backpropagation
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {total_loss / len(data_loader)}')


from torch.utils.data import DataLoader, TensorDataset

data_tensor = torch.Tensor(tensorData) 
data_tensor = data_tensor.unsqueeze(1)  # Adding channel dimension (1829, 59, 101) -> (1829, 1, 59, 101)
dataset = TensorDataset(data_tensor)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize the model and train
model = LigandFeatExtractor()

# Assuming you have a GPU available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
train(model, data_loader, epochs=10, lr=0.001, device=device)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x173824 and 5959x512)

In [ ]:
import torch
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Function to extract features from test data and reduce dimensions
def evaluate(model, data_loader, device='cpu'):
    model.eval()  # Set model to evaluation mode
    all_features = []

    with torch.no_grad():  # Disable gradient computation for evaluation
        for data in data_loader:
            data = data[0].to(device)
            features = model(data)  # Extract features
            all_features.append(features.cpu().numpy())  # Collect features (moving to CPU for sklearn)

    all_features = np.concatenate(all_features, axis=0)  # Combine all extracted features into one array
    
    pca = PCA(n_components=2)  # Reduce to 2D for visualization
    pca_result = pca.fit_transform(all_features)

    # plot results
    plt.figure(figsize=(10, 8))
    plt.scatter(pca_result[:, 0], pca_result[:, 1], s=5, cmap='viridis')
    plt.title("PCA of Feature Representations")
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.show()

test_data_tensor = torch.Tensor(testData)  # Replace with your actual test data
test_data_tensor = test_data_tensor.unsqueeze(1)  # Ensure correct shape (batch, 1, 59, 101)
test_dataset = TensorDataset(test_data_tensor)
test_data_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize the model (already trained)
model = LigandFeatExtractor()

# Evaluate the model
evaluate(model, test_data_loader, device=device)
